In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')


Mounted at /content/gdrive


#Using Multinomial Naive Bayes 


*   Trained on 8 languages
*   Before running download the sentences.csv from https://downloads.tatoeba.org/exports/



In [ ]:
#Importing the dataset
import pandas as pd

df = pd.read_csv('/content/gdrive/My Drive/Data/sentences.csv',
                            sep='\t', 
                            encoding='utf8', 
                            index_col=0,
                            names=['lang','text'])



In [ ]:
filt = [True if 40<=len(s)<=500 else False for s in df['text']]
df = df[filt]

# We will train this on only 8 languages - 
lang_filter = ['deu', 'eng', 'fra', 'ita', 'por', 'spa', 'ara', 'ben']

def clean_data2(data,langlist):
  '''
  To filter the sentences only from lang_filter
  '''
  data = data.loc[data['lang'].isin(langlist)]
  return data

df_new = clean_data2(df,lang_filter)

#Trimming the dataset 
n = 10000
df_red = df_new.groupby('lang').apply(lambda x: x.sample(min(n,len(x)))).reset_index(drop=True)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.feature_extraction.text import CountVectorizer


cnt = CountVectorizer(analyzer = 'char',ngram_range=(2,2))

pipeline = Pipeline([
   ('vectorizer',cnt),  
   ('model',MultinomialNB())
])

# Seperating into X and Y

X = df_red['text']
Y = df_red['lang']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.30, random_state=42)

pipeline.fit(X_train,Y_train)
Y_pred = pipeline.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(Y_test,Y_pred)
print('Test Accuracy - MultinomialNB: ',accuracy)

Test Accuracy - MultinomialNB:  0.9935311633637951


In [ ]:
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

         ara       1.00      1.00      1.00      1390
         ben       1.00      1.00      1.00       100
         deu       1.00      1.00      1.00      2977
         eng       1.00      1.00      1.00      2953
         fra       1.00      1.00      1.00      3026
         ita       0.99      0.99      0.99      3068
         por       0.99      0.99      0.99      2956
         spa       0.98      0.99      0.99      3008

    accuracy                           0.99     19478
   macro avg       0.99      0.99      0.99     19478
weighted avg       0.99      0.99      0.99     19478

